In [1]:
%load_ext autoreload
%autoreload 2

from graph import Graph
from nnsight.models import UnifiedTransformer
from transformer_lens import HookedTransformer
from gt_dataset import EAPDataset
from metrics import get_metric
from functools import partial

In [35]:
g = Graph()
g = g.from_json("circuits/greater-than_prob_diff_ig.json")

In [21]:
[g.nodes[f].qkv_inputs for f in g.nodes.keys() if "a4." in f]

[None, None, None, None, None, None, None, None, None, None, None, None]

In [6]:
model = HookedTransformer.from_pretrained("gpt2-small", device="cuda")
model.cfg.use_split_qkv_input = True
model.cfg.use_attn_result = True
model.cfg.use_hook_mlp_in = True

dataset = EAPDataset("greater-than-gpt2.csv")
dataloader = dataset.to_dataloader(16)
metric_fn = get_metric("prob_diff", "greater-than", model.tokenizer, model)

/home/aaron/miniconda3/envs/othello/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer


In [22]:
from evaluate_graph import evaluate_graph, evaluate_baseline

baseline = evaluate_baseline(model, dataloader, partial(metric_fn, loss=False, mean=False)).mean().item()
results = evaluate_graph(model, g, dataloader, partial(metric_fn, loss=False, mean=False), prune=False).mean().item()

  5%|▍         | 3/63 [00:00<00:02, 25.47it/s]

100%|██████████| 63/63 [00:02<00:00, 29.79it/s]


ValueError: Invalid node: Node(input, in_graph: True) of type <class 'graph.InputNode'>

In [36]:
from evaluate_graph import evaluate_area_under_curve

results = evaluate_area_under_curve(model, g, dataloader, partial(metric_fn, loss=False, mean=False),
                                    prune=False, node_eval=False)
results

  0%|          | 0/63 [00:00<?, ?it/s]

100%|██████████| 63/63 [00:02<00:00, 31.05it/s]


Computing results for 0.1% of edges (N=32)


100%|██████████| 63/63 [00:03<00:00, 20.29it/s]


Computing results for 0.2% of edges (N=64)


...

Computing results for 50.0% of edges (N=16245)


100%|██████████| 63/63 [00:02<00:00, 24.83it/s]


Computing results for 100% of edges (N=32491)


100%|██████████| 63/63 [00:02<00:00, 31.46it/s]


(1.0871138469119255, 0.9912501536188877)

In [8]:
print(f"Faithfulness: {results / baseline}")

Faithfulness: 0.512998701440054
